<a href="https://colab.research.google.com/github/anodyne99/Weapon-Detection-Yolov5/blob/main/YoloV5_Weapon_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### REQUIRED INITIAL STEPS

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -q -U roboflow
%pip install -r requirements.txt

import torch
import os
from IPython.display import Image, clear_output


## Purely to check to make sure you're on a GPU. Training and detection performance is suboptimal without it.
print(f"Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

#### REQUIRED FOR FRESH TRAINING AND ANY TIME A LOCAL DETECTION ISN'T BEING EXECUTED

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="mRU23tAr1sKN2UOIokAG")
project = rf.workspace("umkc-490-spring-2022").project("weapon-detection-nicpz")
dataset = project.version(2).download("yolov5")

### IF STARTING A FRESH TRAINING RUN

In [ ]:
## Counterintuitively, ultralytics recommends higher batch size, at least 64.
## Also, the training automatically stops if loss is plateauing for 100 epochs.

!python train.py --img 736 --batch 64 --epochs 500 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')
files.download('./runs/train/exp/weights/last.pt')

### FOR IMPORTING PRETRAINED MODELS

#### PRETRAINED DEMO MODEL

In [ ]:
#If you're coming from the previous step
#Otherwise ignore this

!wget -P /content/yolov5/weights

/content/yolov5/weights: Scheme missing.


#### LOCAL WEIGHT MODEL UPLOAD

In [ ]:
#Creating the directory
%mkdir /content/yolov5/weights

#modify the first part of path to wherever you have placed the best.pt in your drive
%cp /content/drive/My\ Drive/Colab\ project/best.pt /content/yolov5/weights

### DETECTION

YOLOv5 has some quirks where the model expects something in the same size as the training data. It also wants it in a 1:1 ratio, which may require some cropping and resizing if you're using a local model.

#### LOCAL MODEL


In [ ]:
import shutil
localFile = input("Enter full local file path here:")
%mkdir /content/yolov5/test/
shutil.copytree(localFile, '/content/yolov5/test/images')
# %cp -av '/content/drive/MyDrive/Colab project/obfuscation pistols' /content/yolov5/test/images

!python detect.py --weights /content/yolov5/weights/best.pt --img 736 --conf 0.1 --source /content/yolov5/test/images/

In [ ]:
## Useful for saving the detection images for later review

shutil.copytree('/content/yolov5/runs/detect/exp', '/content/drive/MyDrive/Colab project/detections')

'/content/drive/MyDrive/Colab project/detections'

#### VIDEO DETECTION

In [ ]:
## You may use a direct path, a youtube url, a direct http stream, or 0 for webcam

#inputVideoPath = input("Enter video path here:")
#youtube video prefilled for demonstration

!python detect.py --weights /content/yolov5/weights/best.pt --img 736 --conf 0.1 --source '/content/drive/MyDrive/Colab project/New_video.mp4'

#### USING PRE-SELECTED DATASET

In [ ]:
!python detect.py --weights /content/yolov5/weights/best.pt --img 736 --conf 0.1 --source {dataset.location}/test/images

### OUTPUT RESULTS

####IMAGES

In [ ]:
import glob

path_start = '/content/yolov5/'

print('Enter detection results path.')
file_name = input('It will be the last line from the previous cell after "saved to":')
path = path_start + file_name + "/"
full_path = path + '*.jpg'

for imageName in glob.glob(full_path):
  display(Image(filename=imageName))
  print("\n")

#### VIDEO

Download the video from the local files on the left. Colab video streaming integration is not fully stable and can bloat ipynb files past allotted memory, causing a runtime crash. The detection bounding boxes will be overlaid on the video.

These will be stored in /content/yolov5/detect/